# Monte Carlo simulation of the Monte Hall problem

In [1]:
using Random # for shuffle!
using StatsBase # for sample

┌ Info: Recompiling stale cache file /home/simoncor/.julia/compiled/v1.2/StatsBase/EZjIG.ji for StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91]
└ @ Base loading.jl:1240


for simplicity, define the two types of items that can be behind a door

In [2]:
const goat = :goat
const car = :car

:car

Define a `mutable struct` (see [here](https://docs.julialang.org/en/v1/manual/types/index.html#Mutable-Composite-Types-1)) to represent a door.

In [3]:
mutable struct Door # mutable b/c we want to open and close the door
    wuts_behind::Symbol # a car or a goat?
    open::Bool # is the door open?
end

door = Door(car, false) # construct a closed door with a car behind it

Door(:car, false)

In [4]:
"""
construct and return an array of three un-opened doors such that: 
* two doors have a goat behind them
* one door has a car behind it
* which door has the car is chosen at uniform random.
"""
function set_up_doors()
    # construct an array of three un-opened doors, one with a car behind it
    doors = [Door(car, false), Door(goat, false), Door(goat, false)]
    # randomly shuffle the doors
    shuffle!(doors)
    return doors
end

doors = set_up_doors()

3-element Array{Door,1}:
 Door(:car, false) 
 Door(:goat, false)
 Door(:goat, false)

In [5]:
"""
return the index of a random door to simulate the contestant picking a door at random
"""
function pick_a_door_at_random()
    return sample(1:3)
end

first_door_id_pick = pick_a_door_at_random()

1

the [enumerate](https://docs.julialang.org/en/v1/base/iterators/#Base.Iterators.enumerate) iterator is useful here.

so is the [continue](https://docs.julialang.org/en/v1/manual/control-flow/) statement for control flow

In [6]:
"""
open a door such that:
* the door that the contestant picked is not opened
* the door with the car is not opened
* if two doors are viable to open, pick one at uniform random
"""
function open_a_door!(doors::Array{Door}, first_door_id_pick::Int)
    ###
    #   determine which doors are viable to open
    ###
    # make an array of viable door IDs to choose from at random
    viable_door_ids = Int64[]
    # loop through the doors...
    for (door_id, door) in enumerate(doors)
        # don't push the door ID that the contestant chose!
        if door_id == first_door_id_pick
            continue
        end
        # don't push the door ID that has the car behind it, that would ruin the game!
        if door.wuts_behind == car
            continue
        end
        # if we made it this far: 
        #  (1) this door was not chosen by the contestant
        #  (2) this door doesn't have a car behind it
        # so, it's a viable door to open
        push!(viable_door_ids, door_id)
    end
    
    ###
    #   of the viable doors, randomly choose one and open it
    ###
    door_id_to_open = sample(viable_door_ids)
    doors[door_id_to_open].open = true
    
    return door_id_to_open # so we know which door was opened
end

open_a_door!(doors, first_door_id_pick)
doors

3-element Array{Door,1}:
 Door(:car, false) 
 Door(:goat, true) 
 Door(:goat, false)

In [7]:
"""
return the index of the door that the contestant must pick if he/she *switches* from his/her initial door choice
to the other unopened door. it is assumed that one the `doors` passed has been opened by the game host and contains
a goat.
"""
function switch_door_pick(doors::Array{Door}, first_door_id_pick::Int64)
    # don't call this function unless one door has been opened
    # since to switch, we need to have the contestant reveal an open door first
    @assert sum([door.open for door in doors]) == 1
    
    # loop through doors.
    # return the ID of the one that (a) has not been opened and (b) was not chosen the first time
    for (door_id, door) in enumerate(doors)
        # don't choose the same door you chose the first time; we're switching!
        if door_id == first_door_id_pick
            continue
        end
        # don't choose the opened door since it has a goat as revealed
        if door.open
            @assert door.wuts_behind == goat # to make sure we opened a door without the car!
            continue
        end
        # if made it this far, the door was not chosen the first time and was not opened 
        return door_id
    end
end

second_door_id_pick = switch_door_pick(doors, first_door_id_pick)

3

In [8]:
"""
return true if the contestant picked the door with the car behind it, false otherwise
"""
contestant_wins_car(doors::Array{Door}, door_id_pick::Int) = doors[door_id_pick].wuts_behind == car

contestant_wins_car(doors, second_door_id_pick)

false

put it all together

In [9]:
"""
simulate one Monte Hall game.

# Arguments
* `switch::Bool`: true if the strategy is for the contestant to switch to the door the host did not open
* `verbose::Bool`: true if you want to print off details of the outcomes
"""
function simulate_Monte_Hall(switch::Bool; verbose::Bool=true)
    doors = set_up_doors()
    door_id_pick = pick_a_door_at_random()
    door_id_opened = open_a_door!(doors, door_id_pick)
    if verbose
        for (i, door) in enumerate(doors)
            println("door $i: $(door.wuts_behind)")
        end
        println("\tcontestant chose door $door_id_pick")
        println("\tgame host opened door $door_id_opened, revealed goat behind it")
    end
    if switch
        door_id_pick = switch_door_pick(doors, door_id_pick)
        verbose ? println("\tcontestant switched to door $door_id_pick") : nothing
    end
    won = contestant_wins_car(doors, door_id_pick)
    if verbose
        won ? println("\t:) contestant won!") : println("\t:( contestant lost")
    end
    return won
end

simulate_Monte_Hall(false; verbose=true)

door 1: goat
door 2: goat
door 3: car
	contestant chose door 2
	game host opened door 1, revealed goat behind it
	:( contestant lost


false

## run simulations

In [10]:
nb_sims = 1000

1000

### strategy: don't switch

In [11]:
nb_wins = 0
for s = 1:nb_sims
    # verbose = false to avoid printing off stuff 1000 times
    nb_wins += simulate_Monte_Hall(false; verbose=false)
end

println("# wins: ", nb_wins)

# wins: 317


### strategy: do switch

In [14]:
nb_wins = 0
for s = 1:nb_sims
    nb_wins += simulate_Monte_Hall(true; verbose=false)
end

println("# wins: ", nb_wins)

# wins: 689
